In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import time
from google.colab import drive
import os
import pickle
from scipy.signal import find_peaks, argrelextrema
from scipy.spatial.distance import cdist

In [2]:
def time_ts(natural_time):
        return int(natural_time/0.01)

In [ ]:
spectral_radii = np.linspace(0, 0.8, 17)
spectral_radii = [str(p) for p in spectral_radii]
spectral_radii.append('0.85')
spectral_radii.append('0.90')
spectral_radii.append('0.95')
randMatrices = ['M', 'Win']

In [ ]:
p = spectral_radii[0]
randMat = randMatrices[0]
p, randMat

('0.0', 'M')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

def save_dictionary(dictionary, file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)
    with open(file_path, 'wb') as file:
        pickle.dump(dictionary, file, protocol=pickle.HIGHEST_PROTOCOL)
    print(f'Dictionary saved to {file_path}')

def load_dictionary(file_path):
    with open(file_path, 'rb') as file:
        loaded_dictionary = pickle.load(file)
    print(f"Data loaded successfully from {file_path}")
    return loaded_dictionary

Mounted at /content/drive


In [ ]:
file_path = f'drive/My Drive/bifurcation_analysis_data/{p}_{randMat}.pkl'
file_path

'drive/My Drive/bifurcation_analysis_data/0.0_M.pkl'

In [ ]:
bifurcation_analysis_loaded = load_dictionary(file_path)

Data loaded successfully from drive/My Drive/bifurcation_analysis_data/0.0_M.pkl


In [ ]:
bundle_of_trajectories, _ = bifurcation_analysis_loaded

In [4]:
def get_global_maxima_along_axis(traj, axis):
    local_maxima = []
    for i in range(1, len(traj) - 1):
        window = traj[i-1:i+2]
        middle_point = traj[i]
        # Find the local maxima in terms of the x coordinate.
        if middle_point[axis] == max(window[0][axis], window[1][axis], window[2][axis]):
            #print(i)
            local_maxima.append((i, middle_point))

        # Find the global maximum.
        max_value = float('-inf')
        global_max_index = None
        for index, state in local_maxima:
            # Check if the current state's third value is greater than the current maximum
            if state[axis] > max_value:
                #print(str(index) +" has "+ str(state) + " > " + str(max_value))
                max_value = state[axis]
                global_max_index = index
        global_max_state = traj[global_max_index]

    return global_max_index, global_max_state

In [5]:
def get_global_minima_along_axis(traj, axis):
    local_minima = []
    for i in range(1, len(traj) - 1):
        window = traj[i-1:i+2]
        middle_point = traj[i]
        # Find the local minima in terms of the specified axis.
        if middle_point[axis] == min(window[0][axis], window[1][axis], window[2][axis]):
            local_minima.append((i, middle_point))

    # Find the global minimum.
    min_value = float('inf')
    global_min_index = None
    for index, state in local_minima:
        # Check if the current state's value on the axis is less than the current minimum
        if state[axis] < min_value:
            min_value = state[axis]
            global_min_index = index
    global_min_state = traj[global_min_index]

    return global_min_index, global_min_state

In [6]:
def RK4step(func, current_state, t, dt):
    k1 = func(t, current_state)
    k2 = func(t + 0.5 * dt, current_state + 0.5 * dt * k1)
    k3 = func(t + 0.5 * dt, current_state + 0.5 * dt * k2)
    k4 = func(t + dt, current_state + dt * k3)
    return current_state + (dt / 6) * (k1 + 2 * k2 + 2 * k3 + k4)

In [7]:
class LorenzSystem:

    def __init__(self):
        self.sigma = 10.0
        self.rho = 28.0
        self.beta = 8.0 / 3.0
        self.state = np.array([1, 1, 1])

    def __call__(self, t, u):
        return np.array([
            self.sigma * (u[1] - u[0]),
            u[0] * (self.rho - u[2]) - u[1],
            u[0] * u[1] - self.beta * u[2]
        ])

    def reinitialise_ic(self):
        self.state = np.array([1, 1, 1])

    def randomise_ic(self):
        self.state = 2 * (np.random.rand(self.state.size) - 0.5)

In [8]:
lorenz_system = LorenzSystem()
lorenz_system.randomise_ic()
dt = 0.01
t_end = 300
num_timesteps = int(t_end / dt)

states = [lorenz_system.state]
t = 0.0
for _ in range(num_timesteps):
    lorenz_system.state = RK4step(lorenz_system, lorenz_system.state, t, dt)
    states.append(lorenz_system.state)
    t += dt

states = np.array(states)

In [9]:
clipped_states = states[time_ts(200):]

In [10]:
def get_flow_direction(traj, axis):
  global_max_L_index, global_max_L_value = get_global_maxima_along_axis(traj, axis)
  global_min_L_index, global_min_L_value = get_global_minima_along_axis(traj, axis)

  direction_vector_wing_R = traj[global_max_L_index+1] - global_max_L_value
  direction_vector_wing_L = traj[global_min_L_index+1] - global_min_L_value

  basic_direction_wing_R = np.sign(direction_vector_wing_R)
  basic_direction_wing_L = np.sign(direction_vector_wing_L)
  return (basic_direction_wing_R, basic_direction_wing_L)

In [11]:
(lorenz_direction_wing_R, lorenz_direction_wing_L) = get_flow_direction(clipped_states, 0)

In [ ]:
preserved_orientation = []
for i, trajectories in enumerate(bundle_of_trajectories):
    flow_directions = [(lorenz_direction_wing_R, lorenz_direction_wing_L)]
    for t in trajectories:
        flow_directions.append(get_flow_direction(t, 0))
    print(i)


    if all(np.allclose(flow_directions[0], fd) for fd in flow_directions):
        preserved_orientation.append(True)
    else:
        preserved_orientation.append(False)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [ ]:
preserved_orientation

[True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True]

# CHECK ORIENTATION OF EVERY ATTRACTOR

In [ ]:
file_path = f'drive/My Drive/bifurcation_analysis_data'
orientation_preservation_data_M = {}
orientation_preservation_data_Win = {}

# Function to calculate orientation preservation for each set of trajectories
def analyze_orientation_preservation(bundle_of_trajectory_simulations):
    preserved_orientation_list = []

    for i, trajectories in enumerate(bundle_of_trajectory_simulations):
        flow_directions = []

        # Calculate flow directions for each trajectory
        for t in trajectories:
            flow_directions.append(get_flow_direction(t, 0))  # Assuming axis 0

        # Check if orientation is preserved for the trajectory
        initial_flow_direction = flow_directions[0]
        preserved_orientation = all(np.allclose(initial_flow_direction, fd) for fd in flow_directions)

        preserved_orientation_list.append(preserved_orientation)

    return preserved_orientation_list

# Process each file in the directory
for filename in os.listdir(file_path):
    # Extract parameters from filename
    p, randMat = filename.rstrip('.pkl').split('_')

    # Ensure dictionary keys exist for storing orientation data
    if p not in orientation_preservation_data_M:
        orientation_preservation_data_M[p] = []
    if p not in orientation_preservation_data_Win:
        orientation_preservation_data_Win[p] = []

    # Load data
    full_path = os.path.join(file_path, filename)
    loaded_data_for_analysis = load_dictionary(full_path)
    bundle_of_trajectory_simulations, _ = loaded_data_for_analysis

    # Perform orientation preservation analysis
    preserved_orientation_list = analyze_orientation_preservation(bundle_of_trajectory_simulations)

    # Store results based on randMat value ('M' or 'Win')
    if randMat == 'M':
        orientation_preservation_data_M[p].append(preserved_orientation_list)
    elif randMat == 'Win':
        orientation_preservation_data_Win[p].append(preserved_orientation_list)

# Calculate percentage of trajectories with preserved orientation
def calculate_preserved_percentage(orientation_data):
    total_trajectories = sum(len(orientations) for orientations in orientation_data.values())
    preserved_count = sum(sum(orientations) for orientations in orientation_data.values())
    return (preserved_count / total_trajectories) * 100

# Calculate the percentages
percentage_preserved_M = calculate_preserved_percentage(orientation_preservation_data_M)
percentage_preserved_Win = calculate_preserved_percentage(orientation_preservation_data_Win)

print(f"Percentage of trajectories with preserved orientation for 'M': {percentage_preserved_M:.2f}%")
print(f"Percentage of trajectories with preserved orientation for 'Win': {percentage_preserved_Win:.2f}%")
